In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir "/content/drive/MyDrive/roberta-checkpoints"
!mkdir "/content/drive/MyDrive/roberta-checkpoints/results"

mkdir: cannot create directory ‘/content/drive/MyDrive/roberta-checkpoints’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/roberta-checkpoints/results’: File exists


In [ ]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached datasets-2.11.0-py3-none-any.whl (468 kB)
  Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached multiprocess-0.70.14-py39-none-any.whl (132 kB)
  Using cached aiohttp-3.8.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached huggingface_hub-0.13.4-py3-none-any.whl (200 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached xxhash-3.2.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (212 kB)
  Using cached tokenizers-0.13.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 15.4 MB/s eta 0:00:00


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score

# Check if a GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the dataset
dataset = load_dataset("imdb")

# Split the dataset into training and validation sets
train_dataset = dataset["train"]
val_dataset = dataset["test"]

# Load the pre-trained RoBERTa tokenizer and model
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

# Define the function to tokenize the input text and convert it to PyTorch tensors
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Preprocess the training and validation datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)


output_dir = "/content/drive/MyDrive/roberta-checkpoints"

# Define the training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    learning_rate=2e-5,
    logging_dir="./logs",
    logging_steps=500,
    evaluation_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    gradient_accumulation_steps=4,
)

# Define the function to compute the metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=None,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained("/content/drive/MyDrive/roberta-checkpoints/results/")
tokenizer.save_pretrained("/content/drive/MyDrive/roberta-checkpoints/results/")


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

...

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
1000,0.173100,0.159013,0.951160


Step,Training Loss,Validation Loss,Accuracy
1000,0.173100,0.159013,0.951160
2000,0.085300,0.167823,0.956800


('/content/drive/MyDrive/roberta-checkpoints/results/tokenizer_config.json',
 '/content/drive/MyDrive/roberta-checkpoints/results/special_tokens_map.json',
 '/content/drive/MyDrive/roberta-checkpoints/results/vocab.json',
 '/content/drive/MyDrive/roberta-checkpoints/results/merges.txt',
 '/content/drive/MyDrive/roberta-checkpoints/results/added_tokens.json',
 '/content/drive/MyDrive/roberta-checkpoints/results/tokenizer.json')

In [ ]:
# # #code to resume
# output_dir = "/content/drive/MyDrive/roberta-checkpoints"
# checkpoint = "checkpoint-XXXXX"  # Replace XXXXX with the number of the last saved checkpoint
# model = AutoModelForSequenceClassification.from_pretrained(output_dir + "/" + checkpoint).to(device)
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     data_collator=None,
#     compute_metrics=compute_metrics,
# )
# trainer.train(resume_from_checkpoint=output_dir + "/" + checkpoint)
